In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# FewShotPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


In [2]:
# 기본 PromptTemplate 사용 시
t = PromptTemplate.from_template("What is the capital of {country}.")

t.format(country="korea")

# prompt template의 유효성 테스트 : t.format() - country 를 넣지 않으면 에러 발생

'What is the capital of korea.'

In [ ]:
# 다른 방식의 promptTemplate
t = PromptTemplate(
    template="What is the capital of {country}.",
    # 입력변수전달 - template의 변수가 무엇인지 알려준다.
    input_variables=["country"],
)

t.format(country="korea")

'What is the capital of korea.'

prompt template 사용하는 이유?
prompt template를 디스크에 저장하고 load 할 수 있기 때문.


FewShotPromptTemplate 와 Fewshot Learning 이란?

Fewshot 은 모델에게 예제들을 준다는 뜻과 같다.  
모델에게 어떻게 대답해야 하는 지에 대한 예제를 AI 모델에게 주는 것.  

왜 주는가?
prompt 로 콤마를 써서 구분해줘, 소문자만 써야해 등의 주문을 일일이 전달하는것보다  
내가 원하는 예제를 보여주는 것이 더 성공적!  
왜냐면 모델이 텍스트를 만들기 때문이다.  
즉, prompt 로 명령하는것보다 어떻게 대답해야 하는지에 대한 예제를 제공해 주는 것이 더 나은 방법이다.  




In [ ]:
# 예제 없이 물어보기
chat.predict("what do you know about Korea.")

Korea is a country located in East Asia, divided into two separate sovereign states: North Korea and South Korea. South Korea is a developed country with a high standard of living, while North Korea is a totalitarian regime with a heavily controlled economy and limited freedoms for its citizens.

Korea has a rich cultural heritage, including traditional music, dance, and cuisine. The Korean language is unique and has its own writing system called Hangul.

Korea has a long history of conflict and division, with the Korean War in the 1950s resulting in the current division of the country. The two Koreas have had tense relations ever since, with occasional flare-ups of violence and threats of nuclear war.

South Korea is known for its technological advancements, particularly in the electronics and automotive industries. It is home to companies like Samsung, LG, and Hyundai.

Korean pop culture, known as K-pop, has gained international popularity in recent years, with groups like BTS and B

In [ ]:
# 예제를 형식화 하는 방법
# 예제들
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# 예제 템플릿: example의 question, answer 처럼 동일한 변수를 사용해야 한다.
example_template = """
    Human: {question}
    AI: {answer}
"""
# 예제 prompt 작성
example_prompt = PromptTemplate.from_template(example_template)

# 실제 prompt 만들기
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    # 질문 suffix: 형식화 된 모든 예제 마지막에 나오는 내용.(형식화가 끝나고 진짜 질문(동일한 형식)이 나온다.)
    suffix="question: What do you know about {country}?",
    input_variables=["country"],
) # 우리는 prompt 를 사용해서 각각의 예제들을 형식화 했다.

# format
prompt.format(country="korea")

chain = prompt | chat

chain.invoke({
    "country":"korea"
})

AI: 
        Here is what I know:
        Capital: Seoul
        Language: Korean
        Food: Kimchi and Bulgogi
        Currency: South Korean Won

AIMessageChunk(content='AI: \n        Here is what I know:\n        Capital: Seoul\n        Language: Korean\n        Food: Kimchi and Bulgogi\n        Currency: South Korean Won')

In [4]:
# 채팅 봇을 위한 fewShotPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler

# FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# 예제를 형식화 하는 방법
# 예제들
examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

# 1. 먼저 우리는 example_prompt 를 만들었고, 
# 우리가 지금까지 했던 건 example 목록에 있는 각 예제의 형식을 지정해 주는 것. 
# ChatPromptTemplate를 사용해서 형식을 지정


example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you are a geography expert. you give short answer."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

# chain.invoke({"country": "south korea"})
chain.invoke({"country": "Thailand"})


        I know this:
        Capital: Bangkok
        Language: Thai
        Food: Pad Thai and Tom Yum
        Currency: Thai Baht
        

AIMessageChunk(content='\n        I know this:\n        Capital: Bangkok\n        Language: Thai\n        Food: Pad Thai and Tom Yum\n        Currency: Thai Baht\n        ')

위에서 볼 수 있듯이 examples과 FewShotChatMessagePromptTemplate 를 사용하는 것은 정말 쉽다.  
우리가 답변을 형식화 한 방법은 FewShotChatMessagePromptTemplate를 써서 example_prompt 로 final_prompt의 예제 부분에 넣어준 것 뿐.  

```python
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you are a geography expert. you give short answer."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)
```
나머지는 랭체인이 다 알아서 했다.  
랭체인이 예제를 가지고 알아서 채팅 메세지처럼 형식화를 한 것.  

example_prompt는 예제를 형식화하기 위한 거라는 걸 기억하자!

